In [ ]:
# jupyter_vim

In [ ]:
### run ./scripts/run-spark-streaming.sh at project root

In [ ]:
! ls /opt/spark/jars | grep kafka
! ls /opt/spark/jars | grep avro

In [ ]:
from pyspark.sql import SparkSession

AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID", "user")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY", "User@12345")

# Create a SparkSession
jars = ["org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.4"]
spark = (SparkSession.builder.appName("KafkaStreaming")
     .master("local[1]")
    .config("spark.streaming.stopGracefullyOnShutdown", "true")
    .config("spark.sql.streaming.schemaInference", "true")
    .config("spark.jars.packages", ",".join(jars))
    .config(
        "spark.sql.extensions",
        "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    )
    .config("spark.sql.catalog.default", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.default.catalog", "spark")
    .config("spark.sql.catalog.default.type", "rest")
    .config("spark.sql.catalog.default.uri", "http://iceberg-rest:8181")
    .config("spark.sql.catalog.default.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.default.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.default.warehouse", "s3://warehouse/wh")
    .config("spark.sql.catalog.default.s3.access-key", AWS_ACCESS_KEY_ID)
    .config("spark.sql.catalog.default.s3.secret-key", AWS_SECRET_ACCESS_KEY)
    .getOrCreate()
        )

#    .config("spark.jars", "/opt/spark/jars/kafka-clients-3.4.1.jar") \
spark.sparkContext.setLogLevel("INFO")

# Set up the Kafka configuration
BOOTSTRAP_SERVERS = "kafka-broker:9092"
topic = "message"

spark

In [ ]:
# read from iceberg table using spark
table = spark.read.format("iceberg").load("default.spark.message")
table.show(10, truncate=True)

In [ ]:
from pyiceberg.catalog import load_catalog

catalog = load_catalog(
    "default",
    **{
        "type": "rest",
        "uri": "http://iceberg-rest:8181",
        "s3.endpoint": "http://minio:9000",
        "s3.access-key-id": "admin",
        "s3.secret-access-key": "password",
    },
)

In [ ]:
catalog.list_namespaces()

In [ ]:
catalog.table_exists(f"spark.{topic}")
table = catalog.load_table(f"spark.{topic}")
table.schema().as_arrow()

In [ ]:
table.inspect.history()

In [ ]:
table.current_snapshot()

In [ ]:
table.inspect.manifests()

In [ ]:
table.scan().to_pandas()

In [ ]:
import boto3

s3_client = boto3.client(
    "s3",
    endpoint_url="http://minio:9000",
    aws_access_key_id="admin",
    aws_secret_access_key="password",
    region_name="us-east-1",  # MinIO typically defaults to 'us-east-1'
    config=boto3.session.Config(
        signature_version="s3v4", s3={"addressing_style": "path"}
    ),
)
response = s3_client.head_object(
    Bucket="warehouse",
    Key="spark/text/metadata/snap-2813184513174423103-1-b8e3338d-386c-4376-9c31-6007e6ebbe8f.avro",
)
print(response)

In [ ]:
import pyiceberg

pyiceberg.__version__